In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from gptools.stan import compile_model

import cmdstanpy
cmdstanpy.install_cmdstan()
import nest_asyncio
nest_asyncio.apply()

/root/.cache/pypoetry/virtualenvs/stan-env-Zdpe4cqc-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CmdStan install directory: /root/.cmdstan
CmdStan version 2.33.1 already installed
Test model compilation


In [2]:
import random
from sklearn.preprocessing import LabelEncoder
df = pd.read_csv('../model_data/quality_nj2.csv')
encoder1, encoder2, encoder3, encoder4 = LabelEncoder(), LabelEncoder(), LabelEncoder(), LabelEncoder()
df['RATING_EVENT_CODE'] = encoder1.fit_transform(df['RATING_EVENT'])
df['ENTRY_NAME_CODE'] = encoder2.fit_transform(df['ENTRY_NAME'])
df['PLT_ID_CODE'] = encoder3.fit_transform(df['PLT_ID'])
df['RATER_CODE'] = encoder4.fit_transform(df['RATER'])
plt_coords = df.groupby('PLT_ID_CODE')[['ROW','COL']].mean()
#from scipy.spatial.distance import cdist
#coords = np.array([plt_coords['ROW'].values, plt_coords['COL'].values])
#distances = cdist(coords.T,coords.T, metric='euclidean')
year = pd.to_datetime(df['DATE']).dt.year
num_days_in_year = pd.to_datetime(year.astype(str), format='%Y').dt.is_leap_year * 366 + (~pd.to_datetime(year.astype(str), format='%Y').dt.is_leap_year) * 365
df['DAY_OF_YEAR'] = pd.to_datetime(df['DATE']).dt.dayofyear
# we treat Feb 29 as the same day as Feb 28 for counting Leap Year days
df['DAY_OF_YEAR'] = df['DAY_OF_YEAR'] - pd.to_datetime(df['DATE']).dt.is_leap_year*(pd.to_datetime(df['DATE']).dt.dayofyear >= 60)
#df['TIME_OF_YEAR'] = df['DAY_OF_YEAR']/num_days_in_year
df['ENTRY_CUMCOUNT'] = df.groupby('ENTRY_NAME').cumcount() + 1
encoder5 = LabelEncoder()
df['DAY_CODE'] = encoder5.fit_transform(df['DAY_OF_YEAR'])
df['TIME_OF_YEAR'] = (df['DAY_OF_YEAR']-0.5)/365
df.head()

/tmp/ipykernel_486/1499274158.py:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  year = pd.to_datetime(df['DATE']).dt.year
/tmp/ipykernel_486/1499274158.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['DAY_OF_YEAR'] = pd.to_datetime(df['DATE']).dt.dayofyear
/tmp/ipykernel_486/1499274158.py:17: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['DAY_OF_YEAR'] = df['DAY_OF_YEAR'] - pd.to_datetime(df['DATE']).dt.is_leap_year*(pd.to_datetime(df['DATE']).dt.dayofyear >= 60)
/tmp/ipykernel_486/1499274158.py:17: UserWarning: Could not infer format, so each element will be 

,PLT_ID,TRAIT,DATE,QUALITY,ENTRY_CODE,ENTRY_NAME,COMP,RATER,ROW,COL,...,MONTH_CODE,YEAR_CODE,TEST_LOC,ENTRY_NAME_CODE,PLT_ID_CODE,RATER_CODE,DAY_OF_YEAR,ENTRY_CUMCOUNT,DAY_CODE,TIME_OF_YEAR
0,241,Turf Quality,4/18/18,4,37,NAI-14-132,Columbia River Seed,RMD,17,1,...,1,1,"Adelphia, NJ",62,238,3,108,1,2,0.294521
1,241,Turf Quality,5/10/18,4,37,NAI-14-132,Columbia River Seed,RON,17,1,...,6,1,"Adelphia, NJ",62,238,4,130,2,6,0.354795
2,241,Turf Quality,6/7/18,5,37,NAI-14-132,Columbia River Seed,RON,17,1,...,4,1,"Adelphia, NJ",62,238,4,158,3,9,0.431507
3,241,Turf Quality,7/17/18,5,37,NAI-14-132,Columbia River Seed,RON,17,1,...,3,1,"Adelphia, NJ",62,238,4,198,4,15,0.541096
4,241,Turf Quality,8/15/18,6,37,NAI-14-132,Columbia River Seed,RMD,17,1,...,2,1,"Adelphia, NJ",62,238,3,227,5,18,0.620548


In [3]:
df.groupby('RATER_CODE')['PLT_ID'].count()

RATER_CODE
0     267
1     801
2     801
3    1335
4    3204
5    1602
6    1602
Name: PLT_ID, dtype: int64

In [4]:
df['TIME_OF_YEAR'].nunique()

33

In [5]:
df['DAY_CODE'].nunique()

33

In [6]:
df['RATING_EVENT_CODE'].nunique()

36

In [7]:
df.groupby('DAY_CODE')['TIME_OF_YEAR'].mean()

DAY_CODE
0     0.242466
1     0.291781
2     0.294521
3     0.305479
4     0.321918
5     0.332877
6     0.354795
7     0.376712
8     0.404110
9     0.431507
10    0.442466
11    0.447945
12    0.458904
13    0.530137
14    0.535616
15    0.541096
16    0.595890
17    0.609589
18    0.620548
19    0.639726
20    0.669863
21    0.697260
22    0.708219
23    0.710959
24    0.752055
25    0.773973
26    0.782192
27    0.787671
28    0.809589
29    0.834247
30    0.845205
31    0.856164
32    0.880822
Name: TIME_OF_YEAR, dtype: float64

In [8]:
padding = 5 # adjustable
num_rows = 18
num_cols = 15
pred_N = 100
num_basis_functions = 6
stan_data_time = {"N": len(df["QUALITY"]),                 # Number of responses
             "num_raters":df['RATER'].nunique(),         # Total number of rating events
             "num_unique_days":df['DAY_CODE'].nunique(),
             "num_entries":df['ENTRY_NAME'].nunique(),           # Total number of entries (turfgrass types)
             "num_plots":df['PLT_ID'].nunique(),               # Total number of plots 
             "num_categories": 9,                                       # Total number of rating categories
             "num_events_by_rater": df.groupby('RATER_CODE')['PLT_ID'].count(),
             "smoothing_factor": 8,
             "rater_id": df["RATER_CODE"].values+1,       # rater id for y[n]
             "day_id":df["DAY_CODE"].values+1,            # day id of y[n]
             "entry_id": df["ENTRY_NAME_CODE"].values+1,         # entry of y[n]
             "plot_id": df["PLT_ID_CODE"].values+1,             # plot id of y[n]
             "y": df["QUALITY"].values,                    # the rating value of y[n]
              # new values for fourier data
             "num_rows": num_rows,
             "num_cols": num_cols,
             "num_rows_padded": num_rows + padding,
             "num_cols_padded": num_cols + padding,
             "plot_row" : plt_coords["ROW"].astype(int),
             "plot_col" : plt_coords["COL"].astype(int),
             # values used for time GP
             "time" : df['TIME_OF_YEAR'],
             "c_f" : 1.5,
             "M_f":num_basis_functions, # number of Hilbert Basis functions
             "num_ratings_per_entry": np.max(df.groupby('ENTRY_NAME').count()['PLT_ID']),
             "entry_cumcount": df["ENTRY_CUMCOUNT"],
             "day_id_to_time": df.groupby('DAY_CODE')['TIME_OF_YEAR'].mean(),
             "pred_N": pred_N,
             "pred_time": np.linspace(0,1,pred_N+1)[1:]
            #"pred_rating_event_id": None,
            #"pred_entry_id": None,
            #"pred_plot_id":None,
            #"pred_day":None
              }

In [9]:
model = compile_model(stan_file="fourier_model_time_effect_distinct_entries_v9.stan", force_compile=True)

19:04:25 - cmdstanpy - WARNING - CmdStanModel(compile=...) is deprecated and will be removed in the next major version. The constructor will always ensure a model has a compiled executable.
If you wish to force recompilation, use force_compile=True instead.
19:04:25 - cmdstanpy - INFO - compiling stan file /home/hok/oxb_econ/stan-env/stan_env/src/fourier_model_time_effect_distinct_entries_v9.stan to exe file /home/hok/oxb_econ/stan-env/stan_env/src/fourier_model_time_effect_distinct_entries_v9
19:05:00 - cmdstanpy - INFO - compiled model executable: /home/hok/oxb_econ/stan-env/stan_env/src/fourier_model_time_effect_distinct_entries_v9


In [ ]:
#fit = fourier_model_t.sample(stan_data_time)
fit = model.sample(data=stan_data_time, chains=4, parallel_chains=4, adapt_delta=0.99, max_treedepth=15,
                   refresh=1, iter_warmup=200, iter_sampling=800, save_warmup=True, 
                   output_dir='./model_output/2017kbg_nj', seed=1234567,
                   time_fmt='%Y%m%d')

19:13:32 - cmdstanpy - INFO - CmdStan start processing




chain 1 |                                                                                                 | 00:00 Status




chain 2 |                                                                                                 | 00:00 Status





chain 3 |                                                                                                 | 00:00 Status






chain 4 |                                                                                                 | 00:00 Status





chain 3 |                                                                                                 | 00:00 Status



chain 1 |                                                                                                 | 00:00 Status




chain 2 |                                                                                                 | 00:00 Status






chain 4 |                                                    

In [ ]:
for i in [10,20,30,40,50]:
    x = list(df[df["ENTRY_NAME_CODE"] == i]['TIME_OF_YEAR'])
    sorted_indices = sorted(range(len(x)), key=lambda k: x[k])
    x_sorted = [list(df[df["ENTRY_NAME_CODE"] == i]['TIME_OF_YEAR'])[ind] for ind in sorted_indices]
    y_sorted = [fit.time_effect[:,i].mean(axis=0)[ind] for ind in sorted_indices]
    plt.scatter(x_sorted, y_sorted,s=5)
    plt.plot(x_sorted,y_sorted,linewidth=0.5, label=list(df[df["ENTRY_NAME_CODE"] == i]['ENTRY_NAME'])[0])
plt.title('Time Effect on Turfgrass Quality Ratings')
plt.xlabel('Time of Year')
plt.ylabel('Effect')
plt.legend()
plt.show()

In [ ]:
rand_entries = [i for i in range(stan_data_time["num_entries"])]
random.shuffle(rand_entries)
for i in rand_entries[:5]:
    x = list(df[df["ENTRY_NAME_CODE"] == i]['TIME_OF_YEAR'])
    sorted_indices = sorted(range(len(x)), key=lambda k: x[k])
    x_sorted = [list(df[df["ENTRY_NAME_CODE"] == i]['TIME_OF_YEAR'])[ind] for ind in sorted_indices]
    y_sorted = [fit.time_effect[:,i].mean(axis=0)[ind] for ind in sorted_indices]
    plt.scatter(x_sorted,y_sorted,s=5)
    plt.plot(x_sorted,y_sorted,linewidth=0.5, label=list(df[df["ENTRY_NAME_CODE"] == i]['ENTRY_NAME'])[0])
plt.title('Time Effect on Turfgrass Quality Ratings')
plt.xlabel('Time of Year')
plt.ylabel('Effect')
plt.legend()
plt.show()

In [ ]:
rand_entries = [i for i in range(stan_data_time["num_entries"])]
random.shuffle(rand_entries)
for i in rand_entries[:7]:
    x = list(fit.pred_xn.mean(axis=0)+0.5)
    y = fit.pred_time_effect[:,i].mean(axis=0)
    #plt.scatter(x,y,s=5)
    plt.plot(x,y,linewidth=0.75, label=list(df[df["ENTRY_NAME_CODE"] == i]['ENTRY_NAME'])[0])
plt.title('Time Effect on Turfgrass Quality Ratings')
plt.xlabel('Time of Year')
plt.ylabel('Effect')
plt.legend()
plt.show()

In [ ]:
fit.pred_time_effect[:,1]

In [ ]:
PLT_ROW = df.groupby('PLT_ID')[['ROW', 'COL']].max()['ROW']
PLT_COL = df.groupby('PLT_ID')[['ROW', 'COL']].max()['COL']
mean_matrix_time = np.zeros((np.max(PLT_ROW)+1, np.max(PLT_COL)+1))
mean_matrix_time[PLT_ROW, PLT_COL] = fit.plot_effect.mean(axis=0)
sns.heatmap(mean_matrix_time[1:,1:], cmap="YlGnBu")

In [ ]:
from scipy.special import softmax

def rsm_probability(y, theta, beta, tau):
    unsummed = np.concatenate(([0], theta - beta - tau))
    probs = softmax(np.cumsum(unsummed))
    return probs[y]
    
def cross_entropy_loss(y_true, y_pred):
    #print(y_pred)
    epsilon = 1e-15  # Small constant to avoid log(0) errors
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)  # Clip predicted probabilities to avoid extreme values
    return -np.log(y_pred[y_true])

tau = list(fit.tau_rsm.mean(axis=0))
#beta = list(fit.beta_rsm.mean(axis=0))
for n in [1]+[random.randint(0,9612) for i in range(5)]:
    #theta = fit.plot_effect.mean(axis=0)[df.loc[n]["PLT_ID_CODE"]+1] + fit.time_effect.mean(axis=0)[df.loc[n]["ENTRY_NAME_CODE"]+1][df.loc[n]["ENTRY_CUMCOUNT"]]
    thetaVec = fit.plot_effect[:,df.loc[n]["PLT_ID_CODE"]] + fit.time_effect[:,df.loc[n]["ENTRY_NAME_CODE"],df.loc[n]["ENTRY_CUMCOUNT"]-1]
    #sns.kdeplot(thetaVec)
    #beta_n = beta[df.loc[n]["RATER_CODE"]]
    tau_n = tau[df.loc[n]["RATER_CODE"]]
    probabs = np.array([[rsm_probability(i,thetaVec[j],0,tau_n) for i in range(10)] for j in range(4000)]).mean(axis=0)
    true_val = df["QUALITY"][n]
    print("Cross Entropy Loss: ",end='')
    print(cross_entropy_loss(true_val, [0]+probabs))
    plt.scatter([i for i in range(1,10)],probabs[1:],s=5)
    plt.plot([i for i in range(1,10)],probabs[1:],label=f"Response {n}, True Value: {true_val}",linewidth=0.5)
    #print(f"Response {n} True value: "+str(df["QUALITY"][n]))
plt.legend()
plt.show()

In [ ]:
fit.tau_rsm.mean(axis=0).shape

In [ ]:
loss = []
tau = fit.tau_rsm.mean(axis=0)
for n in range(9612):
    #theta = fit.plot_effect.mean(axis=0)[df.loc[n]["PLT_ID_CODE"]+1] + fit.time_effect.mean(axis=0)[df.loc[n]["ENTRY_NAME_CODE"]+1][df.loc[n]["ENTRY_CUMCOUNT"]]
    thetaVec = fit.plot_effect[:,df.loc[n]["PLT_ID_CODE"]] + fit.time_effect[:,df.loc[n]["ENTRY_NAME_CODE"],df.loc[n]["ENTRY_CUMCOUNT"]-1]
    #sns.kdeplot(thetaVec)
    tau_n = tau[df.loc[n]["RATER_CODE"]]
    true_val = df["QUALITY"][n]
    probabs_true_val = np.array([rsm_probability(true_val,theta,0,tau_n) for theta in thetaVec]).mean()
    loss.append(-np.log(probabs_true_val))
    if n%961 == 0:
        print(n,end='')
        print(" : mean loss : "+str(np.array(loss).mean()))
print(loss)

In [ ]:
fit.time_effect[:,1].mean(axis=0).shape
fit.time_effect[:,1].shape

In [ ]:
fit.lengthscale_f.mean(axis=0)

In [ ]:
fit.sigma_f.mean(axis=0)

In [ ]:
# make a rater dictionary
r_codes = df.groupby('RATER')['RATER_CODE'].mean()
rater_dict = {}
for j in r_codes.index:
    rater_dict[round(r_codes[j])] = j
print(rater_dict)

fit.tau_rsm.mean(axis=0)
for i in [1,2,3]:
    #tau_vals = list(fit.tau_rsm.mean(axis=0)[i])
    plt.scatter([i for i in range(1,10)],fit.tau_rsm.mean(axis=0)[i],s=5)
    plt.plot([i for i in range(1,10)],fit.tau_rsm.mean(axis=0)[i], label=rater_dict[i],linewidth=0.5)
plt.title("Tau values for various raters")
plt.xlabel("Quality")
plt.ylabel("Tau")
plt.legend()
plt.show()

In [ ]:
# make a rater dictionary
r_codes = df.groupby('RATER')['RATER_CODE'].mean()
rater_dict = {}
for j in r_codes.index:
    rater_dict[round(r_codes[j])] = j
print(rater_dict)

fit.tau_rsm.mean(axis=0)
for i in [4,6]:
    #tau_vals = list(fit.tau_rsm.mean(axis=0)[i])
    plt.scatter([i for i in range(1,10)],fit.tau_rsm.mean(axis=0)[i],s=5)
    plt.plot([i for i in range(1,10)],fit.tau_rsm.mean(axis=0)[i], label=rater_dict[i],linewidth=0.5)
plt.title("Tau values for various raters")
plt.legend()
plt.show()

In [ ]:
import math
r_codes = df.groupby('RATER')['RATER_CODE'].mean()
rater_dict = {}
for j in r_codes.index:
    rater_dict[round(r_codes[j])] = j
print(rater_dict)

In [ ]:
fit.intercept_f.mean(axis=0)

In [15]:
import pickle
with open('model_fit_v8.pkl','rb') as inp:
    fit_v8 = pickle.load(inp)

UnsupportedOperation: read

In [ ]:
fig, ax = plt.subplots(figsize=(10, 14))
e_codes = df.groupby('ENTRY_NAME')['ENTRY_NAME_CODE'].mean()
entry_dict = {}
for j in e_codes.index:
    entry_dict[round(e_codes[j])] = j
intercept_f = list(fit.intercept_f.mean(axis=0))

# Generate x-axis values and corresponding entry names
y = range(len(intercept_f))
entry_names = [entry_dict[i] for i in y]

entry_names_sorted = sorted(entry_names, key = lambda x : intercept_f[entry_names.index(x)])
intercept_f_sorted = sorted(intercept_f)

bar = ax.barh(y, intercept_f_sorted)

# Set the x-axis tick labels to the entry names
ax.set_yticks(y, entry_names_sorted)

# Add labels and title
ax.set_ylabel('Entry')
ax.set_xlabel('Mean Intercept')
ax.set_title('Mean Intercept for Each Entry')

In [ ]:
fit.time_effect.mean(axis=0).mean(axis=1)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 14))
e_codes = df.groupby('ENTRY_NAME')['ENTRY_NAME_CODE'].mean()
entry_dict = {}
for j in e_codes.index:
    entry_dict[round(e_codes[j])] = j
intercept_f = list(fit.time_effect.mean(axis=0).mean(axis=1))

# Generate x-axis values and corresponding entry names
y = range(len(intercept_f))
entry_names = [entry_dict[i] for i in y]
entry_names_sorted = sorted(entry_names, key = lambda x : intercept_f[entry_names.index(x)])
intercept_f_sorted = sorted(intercept_f)
bar = ax.barh(y, intercept_f_sorted)

# Set the x-axis tick labels to the entry names
ax.set_yticks(y, entry_names_sorted)

# Add labels and title
ax.set_ylabel('Entry')
ax.set_xlabel('Mean Time Effect Recorded')
ax.set_title('Mean Time Effect Recorded for Each Entry')

In [ ]:
idx = pd.DatetimeIndex(["2/28/2019 10:00:00+00:00",
                        "2/29/2020 11:00:00+00:00"])
idx.dayofyear